In [1]:
import os
import scipy.sparse as sps
import numpy as np


def rowSplit (rowString):
    
    split = rowString.split(",")#occhio che .csv splitta con virgola prof con ::
    split[3] = split[3].replace("\n","")
    
    split[0] = int(split[0])#userID
    split[1] = int(split[1])#filmID
    split[2] = float(split[2])#rating
    split[3] = int(split[3])#timestamp
    
    result = tuple(split)
    return result

URM_file = open(r"C:\Users\aless\PYTHON_NOTEBOOKS\RECOMMENDER_SYSTEMS\movies100krates\ratings.csv", 'r')
URM_file.seek(0)
URM_tuples = []
numberInteractions=0
#ho 100836 voti di  610 utenti su 9724 diversi filmz
URM_file.readline()

#for _ in 10:   #rende esecuzione + easy se necessario con solo 10 ratings
#   URM_file.readline()   
#   numberInteractions+=1
for line in URM_file:
    URM_tuples.append(rowSplit (line))
    numberInteractions+=1
#print(len(URM_tuples))
userList, itemList, ratingList, timestampList = zip(*URM_tuples)
userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)
timestampList = list(timestampList)
print("Num utenti: ",len(set(userList)))
print("Num filmz: ",len(set(itemList)))
print("Num voti: ",len(ratingList))


Num utenti:  610
Num filmz:  9724
Num voti:  100836


In [2]:
print("Statistiche generiche: ")
userList_unique = list(set(userList))
itemList_unique = list(set(itemList))
numUsers = len(userList_unique)
numItems = len(itemList_unique)

print("Number of interactions: ",numberInteractions)#<--compelxity min usare questo
print("N users: ",numUsers)
print ("Number of items\t {}, Number of users\t {}".format(numItems, numUsers))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemList_unique), max(userList_unique)))
print ("Average interactions per user {:.2f}".format(numberInteractions/numUsers))
print ("Average interactions per item {:.2f}\n".format(numberInteractions/numItems))
print ("Sparsity {:.2f} %".format((1-float(numberInteractions)/(numItems*numUsers))*100))

    

Statistiche generiche: 
Number of interactions:  100836
N users:  610
Number of items	 9724, Number of users	 610
Max ID items	 193609, Max Id users	 610

Average interactions per user 165.30
Average interactions per item 10.37

Sparsity 98.30 %


In [3]:
#crea la COO matrix
URM_all = sps.coo_matrix((ratingList, (userList, itemList)))#costruttore(elementi,(x,y))
URM_all.tocsr()
#print(URM_all)
#print("matrice formato coo, con solo elem !=0 e loro coord: ")
#print(URM_all)

#print("matrice formato normale, detta 'densa' ")
#print(URM_all.todense())

itemPopularity = (URM_all>0).sum(axis=0) #per ogni item somma tutti i voti (se val >0)(axis=0 means: per ogni colonna fai la somma)
itemPopularity=np.array(itemPopularity).squeeze()#così lo rende davvero un array sennò è una matrice monodimensionale
print("Popolarità totale, NB tanti saranno a 0")
itemPopularity


Popolarità totale, NB tanti saranno a 0


array([  0, 215, 110, ...,   0,   0,   1], dtype=int32)

In [4]:
print("Number of items w/o ratings:(SONO UN BOTTO)")
print(len(itemPopularity))#la dimensione è data dall'item id. film sono 193610 di cui 7k votati
print(np.count_nonzero(itemPopularity==0))#conta gli zero


Number of items w/o ratings:(SONO UN BOTTO)
193610
183886


In [5]:
print("Definisco split train/test e creo mia ooMatrix train")

train_test_split = 0.80 #fa maschera 80% train 20 test
numInteractions = URM_all.nnz
train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])
test_mask = np.logical_not(train_mask)
#train_mask

userList = np.array(userList)
itemList = np.array(itemList)
ratingList = np.array(ratingList)

URM_train = sps.coo_matrix((ratingList[train_mask], (userList[train_mask], itemList[train_mask])))
URM_train = URM_train.tocsr()
URM_train

URM_test = sps.coo_matrix((ratingList[test_mask], (userList[test_mask], itemList[test_mask])))
URM_test = URM_test.tocsr()
URM_test

Definisco split train/test e creo mia ooMatrix train


<611x193580 sparse matrix of type '<class 'numpy.float64'>'
	with 20130 stored elements in Compressed Sparse Row format>

In [6]:
#EVALUATION METRICS

def recall(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    return recall_score


def MAP(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])
    return map_score

def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    return precision_score
    

In [7]:
#DEFINISCO RECOMMENDATOR e lo runno nel prox coso , top item recommender

class UnpRecommender(object):
    #applicato 1 volta a tutto il train test per "prendere in input" i train values
    def fit(self, URM_train):
        itemPopularity = (URM_train>0).sum(axis=0)
        itemPopularity = np.array(itemPopularity).squeeze()

        # Ordina elem in base alla popolarità
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
    
    #applicato per ogni utente (equiv di predict nella regression)
    def recommend(self, user_id, at=5):#raccomanda gli at + popolari in tutto train set
    
        recommended_items = self.popularItems[0:at]

        return recommended_items

    

In [8]:
recInstance = UnpRecommender()
recInstance.fit(URM_train)#lo fa qui e siamo apposto per sempre su questo data set

user_id=2
recommended_items = recInstance.recommend(user_id, at=5)
recommended_items#qui ho le items raccomandate dal mio recommender per l'utente user

array([ 356,  296,  318, 2571,  593], dtype=int64)

In [9]:
def evaluate_algorithm(URM_test, recommender_object, at=5):
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    num_eval = 0
    
    for user_id in userList_unique:
        relevant_items = URM_test[user_id].indices
        
        if len(relevant_items)>0:
            recommended_items = recommender_object.recommend(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP)) 
    
    

evaluate_algorithm(URM_test, recInstance)


Recommender performance is: Precision = 0.0954, Recall = 0.0295, MAP = 0.0582


Adesso faccio quello con GLOBAL EFFECTS. Cioè raccomando quelli con highest average ratings senza i bias.


In [10]:
#fa global recommender solo standardizzando sull item.
#User bias serve solo x predirre ratings quindi non lo usa
class GlobalEffRecommender(object):
    def fit(self, URM_train):
        self.URM_train = URM_train
        globalAverage = np.mean(URM_train.data)

        URM_train_unbiased = URM_train.copy()
        URM_train_unbiased.data -= globalAverage

        item_mean_rating = URM_train_unbiased.mean(axis=0)
        item_mean_rating = np.array(item_mean_rating).squeeze()

        self.bestRatedItems = np.argsort(item_mean_rating)
        self.bestRatedItems = np.flip(self.bestRatedItems, axis = 0)


    def recommend(self, user_id, at=5, remove_seen=True):#raccomando togliendo elemnti già visti dagli utenti
        if remove_seen:
            unseen_items_mask = np.in1d(self.bestRatedItems, URM_train[user_id].indices,
                                        assume_unique=True, invert = True)
            unseen_items = self.bestRatedItems[unseen_items_mask]
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.bestRatedItems[0:at]
        return recommended_items


    

In [11]:
geRec=GlobalEffRecommender()
geRec.fit(URM_train)

evaluate_algorithm(URM_test, geRec)#è lento peso


Recommender performance is: Precision = 0.1915, Recall = 0.0465, MAP = 0.1441
